## 航空twitter的資料拿來情緒分析

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D, Bidirectional
from keras.optimizers import RMSprop, adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

import re

Using TensorFlow backend.


In [2]:
tweet=pd.read_csv("airline_twitter_dataset/Tweets.csv", encoding="utf-8")

In [3]:
tweet.head(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


In [4]:
len(tweet)

14640

In [5]:
# 只留下需要的資料
tweet=tweet.drop(columns=['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence'
                   ,'airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord'
                   ,'tweet_created','tweet_location','user_timezone',])
tweet.head(3)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...


In [6]:
# tweet['text2']=tweet['text'].str.replace('@VirginAmerica ','',regex=False)

In [7]:
tweet.head(3)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...


In [8]:
import nltk
# nltk.download()
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer, WordNetLemmatizer, PorterStemmer, LancasterStemmer

# stemmer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
porter=PorterStemmer()  #porter stemming
lancaster=LancasterStemmer()  #lancaster stemming

## 對資料作處理

In [9]:
for text in range(len(tweet)):
    review=str(tweet.loc[text,'text'])
    review=re.sub(r'@[A-Za-z0-9]+','',review)
    review=re.sub('[^a-zA-Z0-9-_.]', ' ', review)
    review=[porter.stem(w) for w in word_tokenize(str(review))]  #這邊是選用porter stemming
#     re.sub(r'@[A-Za-z0-9]+','',df.text[343])
#     review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
    review=' '.join(review)
    tweet.loc[text,'text2']=review
tweet.head(10)

,airline_sentiment,text,text2
0,neutral,@VirginAmerica What @dhepburn said.,what said .
1,positive,@VirginAmerica plus you've added commercials t...,plu you ve ad commerci to the experi ... tacki .
2,neutral,@VirginAmerica I didn't today... Must mean I n...,I didn t today ... must mean I need to take an...
3,negative,@VirginAmerica it's really aggressive to blast...,it s realli aggress to blast obnoxi entertain ...
4,negative,@VirginAmerica and it's a really big bad thing...,and it s a realli big bad thing about it
5,negative,@VirginAmerica seriously would pay $30 a fligh...,serious would pay 30 a flight for seat that di...
6,positive,"@VirginAmerica yes, nearly every time I fly VX...",ye nearli everi time I fli VX thi ear worm won...
7,neutral,@VirginAmerica Really missed a prime opportuni...,realli miss a prime opportun for men without h...
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D",well I didn t but now I DO -D
9,positive,"@VirginAmerica it was amazing, and arrived an ...",it wa amaz and arriv an hour earli . you re to...


In [10]:
all_words=' '.join(tweet['text2'])
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

12951

In [11]:
# 找出最長的句子有幾個字
r_len=list()
for text in range(len(tweet)):
    word=word_tokenize(tweet.loc[text,'text2'])
    l=len(word)
    r_len.append(l)
max_review_len=np.max(r_len)
max_review_len

36

In [12]:
tweet.loc[tweet['airline_sentiment']=='positive', 'label'] = 2
tweet.loc[tweet['airline_sentiment']=='negative', 'label'] = 1
tweet.loc[tweet['airline_sentiment']=='neutral', 'label'] = 0
tweet.head(5)

,airline_sentiment,text,text2,label
0,neutral,@VirginAmerica What @dhepburn said.,what said .,0.0
1,positive,@VirginAmerica plus you've added commercials t...,plu you ve ad commerci to the experi ... tacki .,2.0
2,neutral,@VirginAmerica I didn't today... Must mean I n...,I didn t today ... must mean I need to take an...,0.0
3,negative,@VirginAmerica it's really aggressive to blast...,it s realli aggress to blast obnoxi entertain ...,1.0
4,negative,@VirginAmerica and it's a really big bad thing...,and it s a realli big bad thing about it,1.0


In [13]:
from sklearn import model_selection
X = tweet['text2'].to_numpy()
Y = tweet['label'].to_numpy()
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [14]:
print(len(X_train),len(X_validation))

11712 2928


In [15]:
le = LabelEncoder()
Y_train = le.fit_transform(Y_train).reshape(-1,1)
Y_validation = le.transform(Y_validation).reshape(-1,1)

In [16]:
Y_validation

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [17]:
#對label做one-hot encoding
ohe = OneHotEncoder()
Y_train = ohe.fit_transform(Y_train).toarray()
Y_validation = ohe.transform(Y_validation).toarray()

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [18]:
Y_validation

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [19]:
max_words = num_unique_word
max_len = max_review_len
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(tweet['text2'])

In [20]:
train_seq = tok.texts_to_sequences(X_train)
val_seq = tok.texts_to_sequences(X_validation)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(val_seq_mat.shape)

(11712, 36)
(2928, 36)


In [31]:
#建立LSTM模型

model = Sequential()
model.add(Embedding(max_words+1, 300, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True,dropout=0.2, recurrent_dropout=0.2)))# 如果要加兩層，return_sequences=True
# model.add(Dense(128, activation='relu',name='d1'))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(128, activation='relu',name='d2'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',name='d3'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 36, 300)           3885600   
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, 36, 300)           0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 36, 256)           439296    
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 256)               394240    
_________________________________________________________________
d2 (Dense)                   (None, 128)               32896     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
d3 (Dense)                   (None, 128)               16512     
__________

In [32]:
model_fit = model.fit(train_seq_mat,Y_train,batch_size=128,epochs=40,
                      validation_data=(val_seq_mat,Y_validation))

Train on 11712 samples, validate on 2928 samples
Epoch 1/40
11712/11712 [==============================] - 14s 1ms/step - loss: 0.7588 - acc: 0.6705 - val_loss: 0.6515 - val_acc: 0.7391
Epoch 2/40
11712/11712 [==============================] - 12s 993us/step - loss: 0.5605 - acc: 0.7751 - val_loss: 0.5687 - val_acc: 0.7818
Epoch 3/40
11712/11712 [==============================] - 12s 995us/step - loss: 0.4564 - acc: 0.8280 - val_loss: 0.5663 - val_acc: 0.7927
Epoch 4/40
11712/11712 [==============================] - 12s 992us/step - loss: 0.3874 - acc: 0.8590 - val_loss: 0.7975 - val_acc: 0.7845
Epoch 5/40
11712/11712 [==============================] - 12s 992us/step - loss: 0.3385 - acc: 0.8780 - val_loss: 0.6427 - val_acc: 0.7903
Epoch 6/40
11712/11712 [==============================] - 12s 993us/step - loss: 0.2878 - acc: 0.8973 - val_loss: 0.6238 - val_acc: 0.7715
Epoch 7/40
11712/11712 [==============================] - 12s 993us/step - loss: 0.2592 - acc: 0.9106 - val_loss: 0.662